In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import re
from pprint import pprint
import db_mysql as db_mysql

In [51]:
import MySQLdb

hostname = "localhost"
username = "root"
password = ""
db_name  = "drama_ojol"

# Open database connection
db = MySQLdb.connect(hostname,username,password,db_name)
db.set_character_set('utf8')

# prepare a cursor object using cursor() method
cursor = db.cursor()
cursor.execute('SET NAMES utf8;')
cursor.execute('SET CHARACTER SET utf8;')
cursor.execute('SET character_set_connection=utf8;')

def executeSql(sql):
    return cursor.execute(sql);

def fetch(type='all'):
    if type=='all':
        return cursor.fetchall();
    else:
        return cursor.fetchone();
def commit():
    db.commit();

def escapeString(string):
    return MySQLdb.escape_string(string);

# Preprocessing Mentions

In [3]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = re.sub('\.+','.',tweet);
    tweet = re.sub('\,+',',',tweet);
    tweet = re.sub('\"+','"',tweet); #Remove tanda ""
    tweet = tweet.replace("/ ","/");
    tweet = tweet.replace("www. ","www.");
    tweet = re.sub(r'RT', '', tweet);
    tweet = re.sub(r'http\S+', '', tweet, flags=re.MULTILINE);
    tweet = re.sub(r'@([A-Za-z0-9_]+)', '', tweet, flags=re.MULTILINE);
    tweet = re.sub(r'pic.twitter.com/\S+', '', tweet, flags=re.MULTILINE);
    tweet = tweet.replace("?"," ? ");
    tweet = tweet.replace("-"," ");
    tweet = tweet.replace("/"," ");
    tweet = tweet.replace("["," ");
    tweet = tweet.replace("]"," ");
    tweet = tweet.replace("."," ");
    tweet = tweet.replace(","," ");
    tweet = re.sub(r'[^a-z0-9.,?# -]', ' ', tweet, flags = re.IGNORECASE|re.MULTILINE)
    tweet = re.sub(" \d+", " ", tweet)
    tweet = re.sub(r'( +)', ' ', tweet, flags = re.IGNORECASE|re.MULTILINE)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

# Read the tweets one by one and process it

In [4]:
dokumen = []
docs, index = [], []
fp = open('tes.csv', 'r')
line = fp.readline()
while line:
    processedTweet = processTweet(line)
    dokumen.append(processedTweet.split(' '))
    line = fp.readline()
fp.close()

In [34]:
datatest = pd.read_csv("data/kategori/datatest.csv",sep=",", header=None)
datatest.columns = ['kategori']

# print(datatest['kategori'][1])
# if(datasets['kategori'][0]=="aplikas"):
#     print("ada")

# Make dataframe containing list of word each category

In [35]:
df_index = pd.DataFrame({
    'kata':datatest['kategori']
})

# Count word accurance in each document

In [36]:
for i in range(len(dokumen)):
    terdeteksi = []
    for kata in datatest['kategori']:
        if kata in dokumen[i]:
            nilai=1
        else:
            nilai=0
        terdeteksi.append(nilai)
        df_index[i+1] = pd.Series(terdeteksi)

In [37]:
df_index.to_csv("data/matriks_datatest.csv", index=False)
# df_tmp = pd.read_csv("data/matriks_datatest.csv",sep=",")
# df_index = df_tmp.drop(['kata'], axis=1)
df_index

,kata,1,2,3,4,5,6,7,8,9,...,3390,3391,3392,3393,3394,3395,3396,3397,3398,3399
0,tarif,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tarifnya,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,murah,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,mahal,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,harga,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,argo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,voucher,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,diskon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ongkos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ongkosnya,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Open the aggregration matrix

In [38]:
category = pd.read_csv("data/aggregasi_matriks.csv",sep=",", header=0)
list_cat = category.drop(['kategori'], axis=1)

In [39]:
list_category = list_cat.values.tolist()
for category in list_category:
    for item in category:
        float(item)

# Assign matrix of each category into variables

In [40]:
cat_harga = []
cat_aplikasi = []
cat_driver = []
cat_penumpang = []
cat_perusahaan = []
cat_harga = np.array(list_category[0])
cat_aplikasi = np.array(list_category[1])
cat_driver = np.array(list_category[2])
cat_penumpang = np.array(list_category[3])
cat_perusahaan = np.array(list_category[4])

# Convert datatest matrix into numpy array

In [41]:
temp = df_index.drop(['kata'], axis=1)
# temp = df_index
temp_datatest = temp.values.tolist()
datatest = np.array(temp_datatest)
# datatest[:,0].shape

In [42]:
temp

,1,2,3,4,5,6,7,8,9,10,...,3390,3391,3392,3393,3394,3395,3396,3397,3398,3399
0,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Fungsi Cosine Similarity

In [43]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

# Categorisasi dengan cosine

In [52]:
import math
total_label = []
total_datatest = np.size(datatest,1)
for i in range(0,total_datatest):
    harga = cos_sim(datatest[:,i], cat_harga)
    aplikasi = cos_sim(datatest[:,i], cat_aplikasi)
    driver = cos_sim(datatest[:,i], cat_driver)
    penumpang = cos_sim(datatest[:,i], cat_penumpang)
    perusahaan = cos_sim(datatest[:,i], cat_perusahaan)
    maximal = max(harga,aplikasi,driver,penumpang,perusahaan)
    if maximal == harga and math.isnan(maximal)==False:
        label = "harga"
    if maximal == aplikasi and math.isnan(maximal)==False:
        label = "aplikasi"
    elif maximal == driver and math.isnan(maximal)==False:
        label = "driver"
    elif maximal == penumpang and math.isnan(maximal)==False:
        label = "penumpang"
    elif maximal == perusahaan and math.isnan(maximal)==False:
        label = "perusahaan"
    elif math.isnan(maximal)==True:
        label = "others"
        
    print(i+1,"=",label,maximal)
    
#     sql = '''UPDATE opinions
#     		 SET cat_result="%s" WHERE id=%d''' % \
#     		 (label,i+1);
#     executeSql(sql);
#     commit();

1 = penumpang 0.8701215571630665
2 = harga 0.3103066836604394
3 = driver 0.9898929393657048
4 = perusahaan 0.4821727535697692
5 = driver 0.4041221002382974
6 = driver 0.9898929393657048
7 = harga 0.33863238032991805
8 = driver 0.5715149550116996
9 = perusahaan 0.4262738440262839
10 = perusahaan 0.4821727535697692
11 = penumpang 0.6152688535266024
12 = penumpang 0.5770687477691996
13 = aplikasi 0.1375408861949216
14 = aplikasi 0.36953446737963064
15 = driver 0.9898929393657048
16 = aplikasi 0.238227803007653
17 = others nan


C:\Users\Brandon\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


18 = driver 0.4949464696828524
19 = aplikasi 0.5008411317516083
20 = perusahaan 0.6818952475051479
21 = driver 0.9898929393657048
22 = perusahaan 0.3409476237525739
23 = aplikasi 0.3129050401563706
24 = penumpang 0.6152688535266024
25 = perusahaan 0.6818952475051479
26 = others nan
27 = driver 0.5715149550116996
28 = penumpang 0.13684404289136093
29 = aplikasi 0.5242887510122971
30 = aplikasi 0.0866399278889315
31 = aplikasi 0.5008411317516083
32 = driver 0.6999600100741737
33 = aplikasi 0.5730037673073569
34 = driver 0.5715149550116996
35 = driver 0.9898929393657048
36 = driver 0.6999600100741737
37 = driver 0.5715149550116996
38 = penumpang 0.6152688535266024
39 = penumpang 0.07900694500032393
40 = others nan
41 = driver 0.5715149550116996
42 = driver 0.4861016670948075
43 = perusahaan 0.6818952475051479
44 = driver 0.0030813789945541054
45 = driver 0.009244136983662318
46 = penumpang 0.6152688535266024
47 = driver 0.6999600100741737
48 = driver 0.5715149550116996
49 = aplikasi 0.238

292 = driver 0.4426935805737587
293 = aplikasi 0.10611180733970595
294 = driver 0.9898929393657048
295 = others nan
296 = driver 0.5715149550116996
297 = perusahaan 0.5021247289246692
298 = perusahaan 0.05643271032077958
299 = others nan
300 = aplikasi 0.5642425368653473
301 = harga 0.010366296258563385
302 = aplikasi 0.40070967096126053
303 = perusahaan 0.6197202241508265
304 = aplikasi 0.23344597724348806
305 = aplikasi 0.09191466409152875
306 = driver 0.6999600100741737
307 = driver 0.9898929393657048
308 = aplikasi 0.5008411317516083
309 = others nan
310 = aplikasi 0.09191466409152875
311 = driver 0.12201637535659438
312 = driver 0.4949464696828524
313 = perusahaan 0.4821727535697692
314 = harga 0.5486235834027787
315 = perusahaan 0.4821727535697692
316 = driver 0.5434781857999811
317 = aplikasi 0.3369049926881063
318 = aplikasi 0.36741213647617055
319 = others nan
320 = driver 0.9898929393657048
321 = aplikasi 0.19320837743750205
322 = driver 0.6999600100741737
323 = aplikasi 0.21

565 = driver 0.9898929393657048
566 = aplikasi 0.1913325664239864
567 = others nan
568 = perusahaan 0.6818952475051479
569 = driver 0.6999600100741737
570 = driver 0.4949464696828524
571 = driver 0.5715149550116996
572 = driver 0.9898929393657048
573 = driver 0.9898929393657048
574 = harga 0.25915743264935165
575 = driver 0.4426935805737587
576 = perusahaan 0.028216354143033435
577 = perusahaan 0.41722344421515606
578 = driver 0.5195975002850193
579 = driver 0.4949464696828524
580 = penumpang 0.07900694500032393
581 = aplikasi 0.3541481605587073
582 = aplikasi 0.3742239920068529
583 = perusahaan 0.4821727535697692
584 = perusahaan 0.3936924047062236
585 = perusahaan 0.3936924047062236
586 = driver 0.6999600100741737
587 = others nan
588 = harga 0.18313791899462253
589 = perusahaan 0.31757156257931696
590 = perusahaan 0.39659654544091283
591 = perusahaan 0.6818952475051479
592 = perusahaan 0.051729984630274006
593 = driver 0.5434781857999811
594 = driver 0.6999600100741737
595 = driver 

...

2245 = harga 0.33863238032991805
2246 = aplikasi 0.09003885501572434
2247 = others nan
2248 = harga 0.5925134700750011
2249 = harga 0.2228753928349057
2250 = others nan
2251 = harga 0.6599875963495982
2252 = others nan
2253 = others nan
2254 = harga 0.6997250701898814
2255 = aplikasi 0.09003885501572434
2256 = penumpang 0.5023649152557929
2257 = driver 0.9898929393657048
2258 = harga 0.07601951365472913
2259 = driver 0.9898929393657048
2260 = driver 0.6999600100741737
2261 = perusahaan 0.05643271032077958
2262 = aplikasi 0.3541481605587073
2263 = others nan
2264 = driver 0.013866204798360484
2265 = driver 0.5715149550116996
2266 = harga 0.290256324334516
2267 = others nan
2268 = harga 0.33863238032991805
2269 = penumpang 0.8701215571630665
2270 = aplikasi 0.08356305019825465
2271 = penumpang 0.8701215571630665
2272 = others nan
2273 = others nan
2274 = harga 0.6599875963495982
2275 = harga 0.6719239200970288
2276 = aplikasi 0.018758094633466636
2277 = driver 0.717390920976203
2278 = ot

2521 = driver 0.5715149550116996
2522 = driver 0.014636549208432515
2523 = driver 0.5715149550116996
2524 = aplikasi 0.09191466409152875
2525 = penumpang 0.6152688535266024
2526 = others nan
2527 = penumpang 0.597724452954421
2528 = driver 0.4949464696828524
2529 = others nan
2530 = others nan
2531 = harga 0.33863238032991805
2532 = others nan
2533 = aplikasi 0.5008411317516083
2534 = driver 0.9898929393657048
2535 = penumpang 0.8701215571630665
2536 = penumpang 0.38913019010091715
2537 = perusahaan 0.4821727535697692
2538 = perusahaan 0.3936924047062236
2539 = penumpang 0.8701215571630665
2540 = others nan
2541 = driver 0.6999600100741737
2542 = driver 0.9898929393657048
2543 = driver 0.5715149550116996
2544 = aplikasi 0.19320837743750205
2545 = perusahaan 0.3936924047062236
2546 = others nan
2547 = harga 0.1832519780212525
2548 = aplikasi 0.08065980963670222
2549 = driver 0.06863421113808432
2550 = others nan
2551 = aplikasi 0.33389408912621305
2552 = driver 0.6999600100741737
2553 =

2797 = perusahaan 0.3936924047062236
2798 = driver 0.5715149550116996
2799 = perusahaan 0.3409476237525739
2800 = aplikasi 0.33389408912621305
2801 = driver 0.6999600100741737
2802 = aplikasi 0.3369049926881063
2803 = aplikasi 0.46785561686516336
2804 = others nan
2805 = penumpang 0.20082959599025826
2806 = others nan
2807 = others nan
2808 = aplikasi 0.08065980963670222
2809 = penumpang 0.8701215571630665
2810 = aplikasi 0.12380342613104876
2811 = others nan
2812 = perusahaan 0.05956786078111662
2813 = harga 0.5306686299168502
2814 = penumpang 0.43506077858153325
2815 = perusahaan 0.08935179015431859
2816 = others nan
2817 = harga 0.4666817048778108
2818 = aplikasi 0.4244472348394987
2819 = aplikasi 0.34655971603067837
2820 = penumpang 0.6152688535266024
2821 = harga 0.1832519780212525
2822 = perusahaan 0.6818952475051479
2823 = others nan
2824 = aplikasi 0.19320837743750205
2825 = others nan
2826 = others nan
2827 = aplikasi 0.12380342613104876
2828 = others nan
2829 = driver 0.98989

3076 = aplikasi 0.0018758090758044241
3077 = driver 0.9898929393657048
3078 = aplikasi 0.378913514696364
3079 = driver 0.5715149550116996
3080 = aplikasi 0.11817599890363548
3081 = harga 0.15549446133529546
3082 = driver 0.9898929393657048
3083 = harga 0.6719239200970288
3084 = aplikasi 0.33389408912621305
3085 = harga 0.6599875963495982
3086 = harga 0.47512196034205706
3087 = others nan
3088 = driver 0.5715149550116996
3089 = driver 0.6999600100741737
3090 = aplikasi 0.238227803007653
3091 = harga 0.6719239200970288
3092 = perusahaan 0.051729984630274006
3093 = harga 0.4666817048778108
3094 = perusahaan 0.08935179015431859
3095 = harga 0.6599875963495982
3096 = others nan
3097 = others nan
3098 = perusahaan 0.7038413007275072
3099 = harga 0.315193403266355
3100 = harga 0.10020754019769311
3101 = driver 0.4949464696828524
3102 = penumpang 0.2840159383753326
3103 = harga 0.38104401641425467
3104 = others nan
3105 = perusahaan 0.4976909566235803
3106 = perusahaan 0.4063629644418026
3107 

3354 = penumpang 0.2840159383753326
3355 = driver 0.4949464696828524
3356 = driver 0.5715149550116996
3357 = driver 0.9898929393657048
3358 = harga 0.133664655781651
3359 = driver 0.6999600100741737
3360 = driver 0.5715149550116996
3361 = driver 0.6275545537406171
3362 = driver 0.0970634322342574
3363 = driver 0.9898929393657048
3364 = driver 0.6999600100741737
3365 = others nan
3366 = driver 0.032354477411419134
3367 = driver 0.6999600100741737
3368 = driver 0.4949464696828524
3369 = harga 0.20524221521921812
3370 = driver 0.5857472341656623
3371 = penumpang 0.20082959599025826
3372 = driver 0.5715149550116996
3373 = aplikasi 0.4363848106989119
3374 = others nan
3375 = perusahaan 0.4976909566235803
3376 = perusahaan 0.4976909566235803
3377 = driver 0.6999600100741737
3378 = perusahaan 0.7038413007275072
3379 = aplikasi 0.5946316068566526
3380 = driver 0.5715149550116996
3381 = driver 0.6999600100741737
3382 = perusahaan 0.4063629644418026
3383 = driver 0.0970634322342574
3384 = aplika